In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import re
import warnings
warnings.simplefilter('ignore')

In [2]:
import sys
sys.path.append("../scripts/")
from data_cleaner import DataCleaner
from util import Util
from pridict import Predict
from preprocessor import Processor

In [3]:
util = Util()
cleaner = DataCleaner()
predictor = Predict()
processor = Processor()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Natnael\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Natnael\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Natnael\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Natnael\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Natnael\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Natnael\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


# News

In [4]:
repo="https://github.com/Nathnael12/prompt-engineering.git"
test=util.read_from_dvc("data/test_news.csv",repo,"test-news-v3",low_memory=False)
train=util.read_from_dvc("data/trainer_news.csv",repo,"train-news-v3",low_memory=False)

# cleaned_df=util.read_from_file('../data/news.csv')

### our first try structure for co:here API

Task: Score news relevance

Domain: {Domain}

Title: {Title}

Description: {
    long text Description
}

Body: {
    long text Body
}

Analyst_Average_Score : {Analyst_Average_Score}

In [5]:
# all_columns=['Domain', 'Title', 'Description', 'Body', 'Link', 'timestamp','Analyst_Rank', 'Reference_Final_Score', 'Analyst_Average_Score']
# useful_columns=['Domain', 'Title', 'Description', 'Body','Analyst_Rank','Analyst_Average_Score']
# drop_columns=['Link','Reference_Final_Score','timestamp']

# all_train=cleaned_df.head(9)[useful_columns]
# test=cleaned_df.tail(1).drop(drop_columns,axis=1)


# tuner=predict(all_train,test,return_value=True)

In [6]:

# all_columns=['Domain', 'Title', 'Description', 'Body', 'Link', 'timestamp','Analyst_Rank', 'Analyst_Average_Score']
# useful_columns=[ 'Description', 'Body','Analyst_Average_Score']
# drop_columns=['Title','Domain', 'Link','Reference_Final_Score','timestamp','Analyst_Rank']

# unique_train=train.drop_duplicates(subset=["Analyst_Average_Score"], keep='last')
# unique_train=unique_train.sample(1)[useful_columns]
# unique_train=train.sample()[useful_columns]
# unique_test = test.drop(drop_columns,axis=1).sample()

# unique_train=unique_train.rename(columns={"Analyst_Average_Score":"Analyst Average Score"})
# unique_test=unique_test.rename(columns={"Analyst_Average_Score":"Analyst Average Score"})

# display(unique_test)
# display(unique_train)

# unique_train_e=pd.DataFrame()



In [11]:
unique_test=test.sample()
unique_train=train.sample()

predictor.predict(unique_train,unique_test) # not fine-tuned
predictor.predict(unique_train,unique_test,model="a0b276d4-adf8-453e-983f-31b8761e8521-ft") # news-tuner: first try
prompt=predictor.predict(unique_train,unique_test, model="c6af2dfd-16ae-4503-9693-6e50dae3861a-ft" ) # tuner: second try
prompt=predictor.predict(unique_train,unique_test, model="acfdd84f-2b81-4ee2-92d5-22ca5ee8f4f0-ft" ) # enhanced-tuner: third try
prompt=predictor.predict(unique_train,unique_test, model="e74ec85a-8e14-4913-83d5-02fe80ac7c4f-ft" ) # tuner-all-in-one-column: final try

print(f"Correct Value: {unique_test.iloc[0,-1]}")

Prediction:  1.28

Prediction:  1.33

Prediction:  1.0

Prediction:  1.33

Prediction:  0.0

Correct Value: 0.0


as we can see from the above results: our las fine tune performs well compared to the others <br>
given example score is 1.33, given test score 0.0, predicted score 0.0

In [12]:
print(prompt)

Task: Generate Analyst Average Score

Description: male, murder, arrest, femal, sap, 2021, 22, 13, elderli, report, 09, crime, 58, cofimvaba, road, stamp, recov, septemb, crime, 9, court, r101, 2021, 09, firearm, robberi, 05, 02, n1, suspect, appear, 14, sap, report, alleg, 42, provinci, street, septemb, magistr, servic, 9, deceas, elizabethan, motorist, male, place, investig, neighbor, bodi, jonker, firearm, bar, murder, suspect, discov, 082, corrupt, set, door, aim4chang, offic, femal, report, 301, eastern, fraud, commit, stamp, road, epesikeni, enquiri, commission, 8, visibl, r101, lifeless, sleep, arrest, 10, 09, know, cofimvaba, lay, cover, 20, appear, twitter, spike, allegedli, hous, s, bed, polic, year, homestead, ear, wednesday, 22, 13, 58, charg, tomorrow, morn, connect, captain, reveng, busi, robberi, 07, 05, south, n1, question, accord, locat, member, african, sap, motiv, cape, 38, open, watch, recov, 59, yesterday, crime, court, room, hide, eye, foil, 2021, 19, neighbourhoo

# Jobs

In [4]:
job_train_df=pd.read_json("../data/job_description_train.json")
job_test_df=pd.read_json("../data/job_description_test.json")

processed_job_train=job_train_df.copy()
processed_job_test=job_test_df.copy()

processed_job_test=processor.prepare_job_description_text(job_test_df)
processed_job_train=processor.prepare_job_description_text(job_train_df)



In [12]:
trainer=processed_job_train.sample(2)
test=processed_job_test.sample(1)

prompt_job=predictor.extract_entities(trainer,test)
print()
prompt_job=predictor.extract_entities(trainer,test,'a724ac98-2abc-47b7-96b3-a77c3a5eb0f8-ft')

Prediction: 

EXPERIENCE: 2+ years, BS/BA Degree

SKILLS: online advertising, research


You can read the whole documentation here. 
In addition, you can use different tokenizers (or language

Prediction:  with 7+ years of experience in data science or quantitative research

EXPERIENCE: 3+ years, 3+ years

SKILLS: quantitative field, data science, quantitative research, programming, statistical software, lab-based research
